# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [4]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory

In [7]:
ws = Workspace.from_config()

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
url = 'https://github.com/BusolaAnu/CapstoneProject/raw/main/heart_failure_clinical_records_dataset.csv'
df = TabularDatasetFactory.from_delimited_files(url)
dataset = df.to_pandas_dataframe()

In [8]:
experiment_name = 'CapstoneProject'

experiment=Experiment(ws, experiment_name)

In [9]:
run = experiment.start_logging()

In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "UdacityProject"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [11]:
dataset.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [12]:
dataset.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2, random_state=11)
label_column_name = "DEATH_EVENT"

In [14]:
from azureml.train.automl import AutoMLConfig
# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations": 5,
    "primary_metric": "accuracy",
    "verbosity": logging.INFO,
    "enable_stack_ensemble": False,
}


# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    debug_log="automl_errors.log",
    training_data=train,
    label_column_name=label_column_name,
    **automl_settings)

In [15]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output = True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneProject,AutoML_c63c88b0-ddbd-4c44-a55b-60f59685204c,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [16]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [24]:
# !pip show azureml-core

In [18]:
import pandas as pd

In [21]:
# print(pd.__version__)

0.25.3


In [22]:
# !pip install --upgrade pandas==0.25.1

     |████████████████████████████████| 10.5 MB 7.1 MB/s eta 0:00:01/s eta 0:00:01
ERROR: torch-tb-profiler 0.2.1 has requirement pandas>=1.0.0, but you'll have pandas 0.25.1 which is incompatible.
ERROR: raiwidgets 0.7.0 has requirement jinja2==2.11.3, but you'll have jinja2 2.11.2 which is incompatible.
ERROR: datasets 1.8.0 has requirement tqdm<4.50.0,>=4.27, but you'll have tqdm 4.61.2 which is incompatible.
ERROR: azureml-responsibleai 1.33.0 has requirement responsibleai~=0.9.1, but you'll have responsibleai 0.7.0 which is incompatible.
ERROR: azureml-opendatasets 1.33.0 has requirement scipy<=1.4.1,>=1.0.0, but you'll have scipy 1.5.2 which is incompatible.
ERROR: azureml-automl-dnn-nlp 1.33.0 has requirement pandas>=0.25.3, but you'll have pandas 0.25.1 which is incompatible.
ERROR: autokeras 1.0.15 has requirement tensorflow>=2.3.0, but you'll have tensorflow 2.1.0 which is incompatible.
  Attempting uninstall: pandas
    Found existing installation: pandas 0.25.3
    Uninstal

In [23]:
# !pip install --upgrade scikit-learn==0.22.1

     |████████████████████████████████| 7.0 MB 6.7 MB/s eta 0:00:01
ERROR: raiwidgets 0.7.0 has requirement jinja2==2.11.3, but you'll have jinja2 2.11.2 which is incompatible.
ERROR: azureml-responsibleai 1.33.0 has requirement responsibleai~=0.9.1, but you'll have responsibleai 0.7.0 which is incompatible.
ERROR: azureml-automl-dnn-nlp 1.33.0 has requirement pandas>=0.25.3, but you'll have pandas 0.25.1 which is incompatible.
ERROR: azureml-automl-dnn-nlp 1.33.0 has requirement scikit-learn>=0.22.2, but you'll have scikit-learn 0.22.1 which is incompatible.
ERROR: autokeras 1.0.15 has requirement tensorflow>=2.3.0, but you'll have tensorflow 2.1.0 which is incompatible.
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [25]:
best_run, best_model = remote_run.get_output()
best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=1.25, reg_lambda=0.8333333333333334, subsample=0.6, tree_method='auto'))], verbose=False)), ('26', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l1')), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bylevel=1, colsample_bytree=1, eta=0.5, gamma=0.01, max_depth=7, max_leaves=15, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=0.5208333333333334, reg_lambda=2.291666666666667, subsample=1, tree_method='au

In [26]:
#TODO: Save the best model
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneProject,AutoML_c63c88b0-ddbd-4c44-a55b-60f59685204c_32,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [28]:
best_model= best_run.register_model(model_path='outputs/model.pkl',model_name='model_Automl',tags={'training context':'automl'},
                        properties={'Accuracy':best_run.get_metrics()['accuracy']})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [29]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [30]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [39]:
best_run.get_metrics()

{'balanced_accuracy': 0.8496491702741704,
 'precision_score_macro': 0.8800257770846006,
 'average_precision_score_weighted': 0.9141537574090188,
 'weighted_accuracy': 0.9036270418991416,
 'matthews_correlation': 0.7281638445971599,
 'AUC_macro': 0.9104669462481961,
 'precision_score_micro': 0.8790780141843971,
 'precision_score_weighted': 0.8823078526379527,
 'f1_score_macro': 0.8582691789470107,
 'recall_score_macro': 0.8496491702741704,
 'AUC_weighted': 0.9104669462481964,
 'recall_score_micro': 0.8790780141843971,
 'f1_score_micro': 0.8790780141843971,
 'norm_macro_recall': 0.6992983405483406,
 'log_loss': 0.3960727498525687,
 'AUC_micro': 0.9165630815854333,
 'average_precision_score_micro': 0.9194086417081188,
 'f1_score_weighted': 0.8753803023840998,
 'average_precision_score_macro': 0.8934605327400462,
 'recall_score_weighted': 0.8790780141843971,
 'accuracy': 0.8790780141843971,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_c63c88b0-ddbd-4c44-a55b-60f59685204c_

In [50]:
from azureml.core import Environment
env = Environment.get(ws, name = "southcentralus")

Exception: Error retrieving the environment definition. Code: 404
: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "No definitions exist for environment southcentralus",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo": null,
    "additionalInfo": null
  },
  "correlation": {
    "operation": "59de2699bad2ee41ba72f535a9974be4",
    "request": "2e741e596c6eb34d"
  },
  "environment": "southcentralus",
  "location": "southcentralus",
  "time": "2021-08-24T18:28:50.0939121+00:00",
  "componentName": "environment-management"
}

In [35]:
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice

# Register the model to deploy
# model = run.register_model(model_name = "heartfailuremodel", model_path = "heart_failure_model.pkl")

# Combine scoring script & environment in Inference configuration

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1,
                                                      auth_enabled=True)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(workspace = ws,
                       name = "my_web_service",
                       models = [best_model],
                       inference_config = inference_config,
                       deployment_config = deployment_config,
                      overwrite=True)

UserErrorException: UserErrorException:
	Message: There is no run definition or environment specified for the run.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "There is no run definition or environment specified for the run."
    }
}

In [ ]:
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.scoring_uri)
print(service.swagger_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service